In [1]:

import numpy as np # linear algebra
import pandas as pd


# Classifier Libraries
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
import collections


# Other Librariest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, auc, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedShuffleSplit


In [4]:
from anomguard.ml_logic.preprocessing  import preprocessing_smote

In [6]:
data1 = pd.read_csv('../raw_data/creditcard.csv')

In [7]:
df = data1

In [ ]:
# df = data1.copy()
# df['Hour'] = (df['Time'] // 3600) % 24

In [8]:
 #Separate features and target variable
X = df.drop(columns=['Class'])
y = df['Class']

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [10]:
X_test.shape


(56962, 30)

In [12]:
X_train_transformed, X_test_transformed, y_train_smote = preprocessing_smote(X_train=X_train, X_test=X_test, y_train=y_train)

ValueError: A given column is not a column of the dataframe

In [ ]:
# # Initialize RobustScaler
# scaler = RobustScaler()

# # Apply scaling only to 'Time' and 'Amount'
# X_train_smote[['Time', 'Amount']] = scaler.fit_transform(X_train_smote[['Time', 'Amount']])
# X_test[['Time', 'Amount']] = scaler.transform(X_test[['Time', 'Amount']])

In [ ]:
# # Log transform the 'Amount' column to reduce skewness
# X_train_smote['Log_Amount'] = np.log1p(X_train_smote['Amount'])
# X_test['Log_Amount'] = np.log1p(X_test['Amount'])

# # Drop the original 'Amount' column if needed
# X_train_smote.drop(columns=['Amount'], inplace=True)
# X_test.drop(columns=['Amount'], inplace=True)


In [ ]:

# # Apply cyclical transformation
# X_train_smote["Hour_sin"] = np.sin(2 * np.pi * X_train_smote["Hour"] / 24)
# X_train_smote["Hour_cos"] = np.cos(2 * np.pi * X_train_smote["Hour"] / 24)

# X_test["Hour_sin"] = np.sin(2 * np.pi * X_test["Hour"] / 24)
# X_test["Hour_cos"] = np.cos(2 * np.pi * X_test["Hour"] / 24)

In [ ]:
# X_train_smote.drop(columns=["Hour"], inplace=True)
# X_test.drop(columns=["Hour"], inplace=True)

In [ ]:
# X_train_smote.shape

(272941, 32)

In [ ]:
# X_test.shape

(56962, 32)

In [ ]:
model_logreg_prepro1 = LogisticRegression(
    class_weight='balanced',  # Handle imbalance class_weight='balanced'    automatically compensates for class imbalance.

    max_iter=1000,
    random_state=42
)
model_logreg_prepro1.fit(X_train_smote, y_train_smote)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [ ]:
#  Compute Recallfrom
from sklearn.metrics import recall_score, precision_recall_curve, auc

# Predict labels
y_pred = model_logreg_prepro1.predict(X_test)
recall_logreg_prepro1 = recall_score(y_test, y_pred)

# Compute PR AUC (Precision-Recall AUC)
y_probs = model_logreg_prepro1.predict_proba(X_test)[:, 1]
precision, recall_curve, _ = precision_recall_curve(y_test, y_probs)
pr_auc = auc(recall_curve, precision)

# Print results
print(f"Recall(LogReg V1.0): {recall_logreg_prepro1:.4f}")  # Исправлено
print(f"PR AUC(LogReg V1.0): {pr_auc:.4f}")

Recall(LogReg V1.0): 0.9082
PR AUC(LogReg V1.0): 0.7839


In [ ]:
import xgboost as xgb

# Create the XGBoost model
model_xgb_preprov1 = xgb.XGBClassifier(
    objective="binary:logistic",  
    scale_pos_weight=len(y_train_smote[y_train_smote == 0]) / len(y_train_smote[y_train_smote == 1]),  
    
    
    eval_metric="logloss",  # This checks how good the model is (lower is better)
    random_state=42,  # This makes sure we get the same results every time we run the model
    use_label_encoder=False  # This removes a warning message
)

# Train (fit) the model with the training data
model_xgb_preprov1.fit(X_train_smote, y_train_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Make predictions on the test data
y_pred_xgb = model_xgb_preprov1.predict(X_test)

# Calculate Recall
recall_xgb = recall_score(y_test, y_pred_xgb)
# Recall tells us how many positive cases we found correctly

# Get probabilities for PR AUC calculation
y_probs_xgb = model_xgb_preprov1.predict_proba(X_test)[:, 1]
# predict_proba() gives probabilities for both classes
# [:, 1] means we take only the probability for the positive class (1)

# Calculate Precision-Recall Curve
precision, recall_curve, _ = precision_recall_curve(y_test, y_probs_xgb)

# Compute PR AUC (Area Under the Precision-Recall Curve)
pr_auc_xgb = auc(recall_curve, precision)

# Print results
print(f"Recall (XGBoost V1.0): {recall_xgb:.4f}")
# Shows the recall score (higher is better)

print(f"PR AUC (XGBoost V1.0): {pr_auc_xgb:.4f}")
# Shows PR AUC (higher is better, closer to 1.0)

Recall (XGBoost V1.0): 0.8469
PR AUC (XGBoost V1.0): 0.8714


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_recall_curve, auc

# Create the model
model_rf_prepro1 = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    class_weight="balanced",  # Adjust for class imbalance
    random_state=42  # Ensure reproducibility
)

# Train the model
model_rf_prepro1.fit(X_train_smote, y_train_smote)

# Predict on test data
y_pred_rf_prepro1 = model_rf_prepro1.predict(X_test)

# Compute recall
recall_rf_prepro1 = recall_score(y_test, y_pred_rf_prepro1)

# Compute PR AUC
y_probs_rf_prepro1 = model_rf_prepro1.predict_proba(X_test)[:, 1]
precision, recall_curve, _ = precision_recall_curve(y_test, y_probs_rf_prepro1)
pr_auc_rf_prepro1 = auc(recall_curve, precision)

# Print results
print(f"Recall (Random Forest prepro1.0): {recall_rf_prepro1:.4f}")
print(f"PR AUC (Random Forest prepro1.0): {pr_auc_rf_prepro1:.4f}")

Recall (Random Forest prepro1.0): 0.8163
PR AUC (Random Forest prepro1.0): 0.8719


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import recall_score, precision_recall_curve, auc
import numpy as np

# Create the TabNet model with name prepro15
model_tabnet_prepro1 = TabNetClassifier(
    optimizer_params=dict(lr=0.02),  # Learning rate (how fast the model learns)
    seed=42  # Fix the random seed for the same results every time
)

# Convert data to NumPy (TabNet works with NumPy, not Pandas)
X_train_np = X_train_smote.to_numpy()
y_train_np = y_train_smote.to_numpy().ravel()  # Fix the shape issue
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy().ravel()  # Fix the shape issue

# Train the model
model_tabnet_prepro1.fit(
    X_train_np, y_train_np,
    eval_set=[(X_test_np, y_test_np)],  # Check model performance on test data
    eval_metric=['logloss'],  # Use log loss to check errors
    max_epochs=15,  # Train for 100 rounds
    patience=5,  # Stop early if no improvement for 10 rounds
    batch_size=1024,  # Number of examples in each training step
    virtual_batch_size=128,  # For faster learning
    num_workers=0  # Number of CPU cores used (0 means auto)
)

In [ ]:
# Make predictions on test data
y_pred_tabnet_prepro1 = model_tabnet_prepro1.predict(X_test_np)

# Calculate Recall
recall_tabnet_prepro1 = recall_score(y_test_np, y_pred_tabnet_prepro1)
# Recall tells us how many positive cases we found correctly

# Get probabilities for PR AUC calculation
y_probs_tabnet_prepro1 = model_tabnet_prepro1.predict_proba(X_test_np)[:, 1]
# predict_proba() gives probabilities for both classes
# [:, 1] means we take only the probability for the positive class (1)

# Calculate Precision-Recall Curve
precision, recall_curve, _ = precision_recall_curve(y_test_np, y_probs_tabnet_prepro1)

# Compute PR AUC (Area Under the Precision-Recall Curve)
pr_auc_tabnet_prepro1 = auc(recall_curve, precision)

# Print results
print(f"Recall (TabNet prepro1.0): {recall_tabnet_prepro1:.4f}")
print(f"PR AUC (TabNet prepro1.0): {pr_auc_tabnet_prepro1:.4f}")


In [ ]:
print(f"Recall(LogReg V1.0): {recall_logreg_prepro1:.4f}")  # Исправлено
print(f"PR AUC(LogReg V1.0): {pr_auc:.4f}")
print(f"Recall (XGBoost V1.0): {recall_xgb:.4f}")
print(f"PR AUC (XGBoost V1.0): {pr_auc_xgb:.4f}")
print(f"Recall (Random Forest prepro15): {recall_rf_prepro1:.4f}")
print(f"PR AUC (Random Forest prepro15): {pr_auc_rf_prepro1:.4f}")

print(f"Recall (TabNet prepro15): {recall_tabnet_prepro1:.4f}")
print(f"PR AUC (TabNet prepro15): {pr_auc_tabnet_prepro1:.4f}")
